In [35]:
import pandas as pd
import numpy as np

np.random.seed(332)

In [36]:
df = pd.read_csv(r'churn_prediction_data\churn_prediction_data\train.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,551,15806307,Trevisano,720,Spain,Male,38,5,114051.97,2,0,1,107577.29,0
1,6897,15709621,Martin,682,France,Female,54,4,62397.41,1,1,0,113088.60,1
2,4588,15619340,Palmer,672,France,Female,31,5,119903.67,1,1,1,132925.17,0
3,291,15620746,Napolitani,592,Spain,Female,40,4,104257.86,1,1,0,110857.33,0
4,1673,15646372,Yao,753,Spain,Male,42,5,120387.73,1,0,1,126378.57,0


In [37]:
from sklearn.preprocessing import LabelEncoder

df = df.drop(['CustomerId', 'RowNumber', 'Surname'], axis=1)

geo_le = LabelEncoder()
gen_le = LabelEncoder()
df['Geography'] = geo_le.fit_transform(df['Geography'])
df['Gender'] = gen_le.fit_transform(df['Gender'])


In [38]:
from sklearn.model_selection import train_test_split

X, y = df.iloc[:, :-1], df.iloc[:, -1]

# stratified split on lable
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42, stratify=y
)

print(f'y_train ratio {y_train.mean()}')
print(f'y_test ration {y_test.mean()}')

y_train ratio 0.2039179104477612
y_test ration 0.20416666666666666


In [39]:
from sklearn.ensemble import RandomForestClassifier
# from xgboost import XGBClassifier
# from lightgbm import LGBMClassifier
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.svm import SVC

# select model
clf = RandomForestClassifier()
# clf = XGBClassifier()
# clf = LGBMClassifier()
# clf = KNeighborsClassifier()
# clf = SVC()

# train and get result
clf.fit(X_train, y_train)

RandomForestClassifier()

In [40]:
# show accuracy
y_pred = clf.predict(X_test)
print('accuracy: ', (y_pred == y_test).mean())

accuracy:  0.8462121212121212


In [41]:
df_upload = pd.read_csv(r'churn_prediction_data\churn_prediction_data\test.csv')

rownum = df_upload['RowNumber']
df_upload = df_upload.drop(['CustomerId', 'RowNumber', 'Surname'], axis=1)
df_upload['Geography'] = geo_le.transform(df_upload['Geography'])
df_upload['Gender'] = gen_le.fit_transform(df_upload['Gender'])

In [42]:
upload_pred = clf.predict(df_upload)

In [43]:
pd.concat([rownum, pd.Series(upload_pred, name='Exited')], axis=1).to_csv(r'output.csv')

In [45]:
# from sklearn.model_selection import GridSearchCV

# param_grid = [
#   {'n_estimators': [100, 200, 300, 400, 500], 'criterion': ['gini', 'entropy']},
#  ]

# clf = GridSearchCV(RandomForestClassifier(), param_grid)
# clf.fit(X_train, y_train)
# sorted(clf.cv_results_.keys())

['mean_fit_time',
 'mean_score_time',
 'mean_test_score',
 'param_criterion',
 'param_n_estimators',
 'params',
 'rank_test_score',
 'split0_test_score',
 'split1_test_score',
 'split2_test_score',
 'split3_test_score',
 'split4_test_score',
 'std_fit_time',
 'std_score_time',
 'std_test_score']

In [47]:
(clf.predict(X_test) == y_test).mean()

0.8481060606060606

In [61]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=5)

for train_index, test_index in skf.split(X, y):

    X_train, X_test = X.iloc[train_index, :], X.iloc[test_index, :]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    clf = RandomForestClassifier(n_estimators = 500)

    clf.fit(X_train, y_train)
    print((clf.predict(X_test) == y_test).mean())

0.86375
0.87125
0.85875
0.85125
0.85375
